In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import string
import re
import time as t
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
# The goal of this prototype is to fetch the lyrics of a specific song, in preparation for the full script that will iterate through the yearly data.

songdata = pd.read_csv('data/2019top10songs.csv')

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
cookies = { 'sessionid': '123..'}
i = 53 # Set i to the position in the songdata datafrae you would like to examine.
url = 'https://www.azlyrics.com/lyrics/'
artist = songdata['Artist'][i] # artist1 and artist2 are the most common ways azlyrics represents artist names. artist1 removes spaces while artist2 replaces                                  them with a dash.
artist1 = ''.join(artist.split()).lower().replace('\'','').replace('!','').replace('.','').replace('+','').replace('ñ','n').replace('é','e')
artist2 = artist.lower().replace('\'','').replace(' ','-').replace('!','').replace('.','').replace('+','').replace('ñ','n').replace('é','e')
artistl = []
if ', ' in artist: # Handles cases where there are three listed artists.
    artistl.append(artist1[:artist1.rfind(',')])
    artistl.append(artist1[artist1.find(',')+1:artist1.rfind('and')])
    artistl.append(artist1[artist1.rfind('and')+3:])
elif ' and ' in artist: # Handles cases where there are two listed artists.
    artistl.append(artist1[:artist1.rfind('and')])
    artistl.append(artist2[:artist2.rfind('-and')])
    if ' featuring ' in artist: # Handles a case with multiple featured artists.
        artistl.append(artist1[:artist1.rfind('featuring')])
        artistl.append(artist2[:artist2.rfind('featuring')])
    artistl.append(artist1) # Cases where 'and' is in the artist name, ex. 'Tones and I'
elif 'bts' in artist1: # Handles that one k-pop group.
    artistl.append('bangtanboys')
elif ' featuring ' in artist: # Handles cases where there is a featured artist.
    artistl.append(artist1[:artist1.rfind('featuring')])
    artistl.append(artist2[:artist2.rfind('featuring')])
else: # Handles cases where there is one artist (which is most of them)
    artistl.append(artist1)
    artistl.append(artist2)
    artistl.append(artist1[artist1.find('the')+3:]) # Handles a rare case where Wikipedia includes a "the" in the artist name and azlyrics does not.
# The first three entries of the above list are standard single artist titles. The next three handle cases of collaborations between two artists. The next four after that handle collaborations between three artists. The last entry handles a rare case where Wikipedia includes a "the" in the artist name and azlyrics does not.
title = ''.join(songdata['Title'][i].split()).lower().replace('\'','').replace(',','').replace('!','').replace('ñ','n').replace('é','e') # azlyrics seems to represent all song titles in the same way.
titlel = [title, 'a' + title, 'the' + title] # However, there are very rare cases, such as 'Holly Jolly Christmas', where azlyrics includes an 'a' or a 'the' at the beginning of a song and Wikipedia does not. These cases will only be tested if all of the artist configurations fail.
if 'futsal' in title:
    titlel.insert(0,'futsalshuffle')
print(artistl)
print(titlel)

['liluzivert', 'lil-uzi-vert', 'luzivert']
['futsalshuffle', 'futsalshuffle2020', 'afutsalshuffle2020', 'thefutsalshuffle2020']


In [4]:
success = False
for j in range(len(titlel)):
    for k in range(len(artistl)):
        if requests.get(url + artistl[k] + '/' + titlel[j] + '.html').status_code == 200:
            html = requests.get(url + artistl[k] + '/' + titlel[j] + '.html',cookies=cookies,headers=headers).text
            soup = BeautifulSoup(html, 'lxml')
            print(url + artistl[k] + '/' + titlel[j] + '.html')
            success = True
            t.sleep(3)
            break
        t.sleep(3)
    if success: # Escapes the outer loop when URL succeeds without using a second request.
        break
    if j == len(titlel) - 1:
        print('Song ' + songdata['Title'][i] + ' by ' + artist + ' at position ' + str(i) + ' not found!')

https://www.azlyrics.com/lyrics/liluzivert/futsalshuffle.html


In [5]:
artistl

['liluzivert', 'lil-uzi-vert', 'luzivert']

In [6]:
my_text = soup.find('div',attrs={'class': None}).text.replace('\n',' ')[3:] # Extract lyrics text from the azlyrics page and replace newlines with spaces. Also, eliminate the \r tag and space at the beginning.
# try:
#     my_text = my_text[:my_text.rindex("[")-1] # Remove miscellaneous notes at the end of lyrics.
# except ValueError:
#     my_text = my_text
my_text = word_tokenize(my_text)
my_text = [i.lower() for i in my_text if not re.fullmatch('[' + string.punctuation + ']+', i)] #Remove tokens that are just punctuation.

In [7]:
port_stem = PorterStemmer() # The following code will stem the words in the lyrics. This means that, for example, "stop", "stopping", and "stops" will only count as one unique word.

stem_text = []

for i in range(len(my_text)):
    stem_text.append(port_stem.stem(my_text[i]))

print(stem_text)

['yo', 'sing', 'thi', 'shit', 'are', "y'all", 'fuckin', 'dumb', 'etern', 'atak', 'got', 'ta', 'shoot', 'two', 'at', 'her', 'got', 'ta', 'shoot', 'two', 'at', 'her', 'blue', 'cheddar', 'that', 'girl', 'she', 'got', 'with', 'my', 'clone', 'i', 'told', 'that', 'lil', 'girl', 'she', 'got', 'ta', 'just', 'do', 'better', 'are', 'you', 'seriou', 'i', 'swear', 'my', 'money', 'is', 'seriou', 'okay', 'all', 'my', 'check', 'i', 'just', 'be', 'clear', 'it', 'she', 'suck', 'my', 'dick', 'till', 'she', 'get', 'deliri', 'okay', 'jump', 'on', 'the', 'jet', 'so', 'i', "'m", 'clear', 'it', 'okay', 'port', 'but', 'i', "'m", 'not', 'talk', 'new', 'okay', 'smoke', 'that', 'boy', 'just', 'like', 'a', 'cigarett', 'if', 'that', 'boy', 'tri', 'to', 'make', 'a', 'fals', 'move', 'uh', 'i', "'m", 'on', 'some', 'whole', 'other', 'shit', 'i', 'got', 'a', 'whole', "'nother", 'rich', 'i', "'m", 'on', 'a', 'new', 'level', 'i', 'wa', 'fuckin', 'on', 'your', 'bitch', 'she', 'wa', 'ridin', 'on', 'my', 'dick', 'she', 'ai'

In [8]:
unique_count = len(set(stem_text))
unique_count

219

In [9]:
len(my_text)

764